Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-text-dnn/auto-ml-classification-text-dnn.png)

# Explain tree-based models on GPU using GPUTreeExplainer


_**This notebook illustrates how to use shap's GPUTreeExplainer on an Azure GPU machine.**_





Problem: Train a tree-based model and explain the model on an Azure GPU machine using the GPUTreeExplainer.

---

## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Run model explainer locally at training time](#Explain)
    1. Apply feature transformations
    1. Train a binary classification model
    1. Explain the model on raw features
        1. Generate global explanations
        1. Generate local explanations
1. [Visualize explanations](#Visualize)
1. [Deploy model and scoring explainer](#Deploy)
1. [Next steps](#Next)

## Introduction
This notebook demonstrates how to use the GPUTreeExplainer on some simple datasets.  Like the TreeExplainer, the GPUTreeExplainer is specifically designed for tree-based machine learning models, but it is designed to accelerate the computations using NVIDIA GPUs.


Make sure you have executed the [configuration](../../../configuration.ipynb) before running this notebook.

Notebook synopsis:

1. Creating an Experiment in an existing Workspace
2. Configuration and remote run with a GPU machine

## Setup

In [ ]:
import logging
import os
import shutil

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.run import Run
from azureml.core.model import Model

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.49.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

As part of the setup you have already created a <b>Workspace</b>. To run the script, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [ ]:
ws = Workspace.from_config()

# Choose an experiment name.
experiment_name = 'gpu-tree-explainer'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil

project_folder = './azureml-shap-gpu-tree-explainer'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('gpu_tree_explainer.py', project_folder)

## Set up a compute cluster
This section uses a user-provided compute cluster (named "gpu-shap-cluster" in this example). If a cluster with this name does not exist in the user's workspace, the below code will create a new cluster. You can choose the parameters of the cluster as mentioned in the comments.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

num_nodes = 1

# Choose a name for your cluster.
amlcompute_cluster_name = "gpu-shap-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6",
                                                           # To use GPUTreeExplainer, select a GPU such as "STANDARD_NC6" 
                                                           # or similar GPU option
                                                           # available in your workspace
                                                           max_nodes = num_nodes)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### Configure & Run

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute target created in previous step
run_config.target = amlcompute_cluster_name

from azureml.core import Environment

environment_name = "shapgpu"
env = Environment(environment_name)

env.docker.enabled = True
env.docker.base_image = None


# Note: this is to pin the pandas and xgboost versions to be same as notebook.
# In production scenario user would choose their dependencies
import pkg_resources
from distutils.version import LooseVersion
available_packages = pkg_resources.working_set
pandas_ver = None
numpy_ver = None
sklearn_ver = None
for dist in list(available_packages):
    if dist.key == 'pandas':
       pandas_ver = dist.version
    if dist.key == 'numpy':
       if LooseVersion(dist.version) >= LooseVersion('1.20.0'):
           numpy_ver = dist.version
       else:
           numpy_ver = '1.21.6'
    if dist.key == 'scikit-learn':
        sklearn_ver = dist.version
pandas_dep = 'pandas'
numpy_dep = 'numpy'
sklearn_dep = 'scikit-learn'
if pandas_ver:
    pandas_dep = 'pandas=={}'.format(pandas_ver)
if numpy_ver:
    numpy_dep = 'numpy=={}'.format(numpy_ver)
if sklearn_ver:
    sklearn_dep = 'scikit-learn=={}'.format(sklearn_ver)

# Note: we build shap at commit 690245 for Tesla K80 GPUs
env.docker.base_dockerfile = f"""
FROM nvidia/cuda:10.2-devel-ubuntu18.04
ENV PATH="/root/miniconda3/bin:${{PATH}}"
ARG PATH="/root/miniconda3/bin:${{PATH}}"
RUN apt-get update && \
apt-get install -y fuse && \
apt-get install -y build-essential && \
apt-get install -y python3-dev && \
apt-get install -y wget && \
apt-get install -y git && \
rm -rf /var/lib/apt/lists/* && \
wget \
https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh && \
mkdir /root/.conda && \
bash Miniconda3-latest-Linux-x86_64.sh -b && \
rm -f Miniconda3-latest-Linux-x86_64.sh && \
conda init bash && \
. ~/.bashrc && \
conda create -n shapgpu python=3.8 && \
conda activate shapgpu && \
apt-get install -y g++ && \
printenv && \
echo "which nvcc: " && \
which nvcc && \
pip install azureml-defaults && \
pip install azureml-telemetry && \
pip install azureml-interpret && \
pip install {pandas_dep} && \
cd /usr/local/src && \
git clone https://github.com/slundberg/shap.git --single-branch && \
cd shap && \
git reset --hard 690245c6ab043edf40cfce3d8438a62e29ab599f && \
mkdir build && \
python setup.py install --user && \
pip uninstall -y xgboost && \
conda install py-xgboost==1.3.3 && \
pip uninstall -y numpy && \
pip install {numpy_dep} && \
pip install {sklearn_dep} && \
pip install chardet \
"""

env.python.user_managed_dependencies = True
env.python.interpreter_path = '/root/miniconda3/envs/shapgpu/bin/python'

from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder, 
                      script='gpu_tree_explainer.py', 
                      compute_target=amlcompute_cluster_name,
                      environment=env) 
run = experiment.submit(config=src)
run

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

In [ ]:
run.get_metrics()

## Download 
1. Download model explanation data.

In [ ]:
from azureml.interpret import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [ ]:
# Get the top k (e.g., 4) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

2. Download model file.

In [ ]:
# Retrieve model for visualization and deployment
from azureml.core.model import Model
import joblib
original_model = Model(ws, 'xgboost_with_gpu_tree_explainer')
model_path = original_model.download(exist_ok=True)
original_model = joblib.load(model_path)

3. Download test dataset.

In [ ]:
# Retrieve x_test for visualization
x_test_path = './x_shap_adult_census.pkl'
run.download_file('x_shap_adult_census.pkl', output_file_path=x_test_path)

In [ ]:
x_test = joblib.load('x_shap_adult_census.pkl')

## Visualize
Load the visualization dashboard

In [ ]:
from raiwidgets import ExplanationDashboard

In [ ]:
from interpret_community.common.model_wrapper import wrap_model
from interpret_community.dataset.dataset_wrapper import DatasetWrapper
# note we need to wrap the XGBoost model to output predictions and probabilities in the scikit-learn format
class WrappedXGBoostModel(object):
    """A class for wrapping an XGBoost model to output integer predicted classes."""

    def __init__(self, model):
        self.model = model

    def predict(self, dataset):
        return self.model.predict(dataset).astype(int)

    def predict_proba(self, dataset):
        return self.model.predict_proba(dataset)

wrapped_model = WrappedXGBoostModel(wrap_model(original_model, DatasetWrapper(x_test), model_task='classification'))

In [ ]:
ExplanationDashboard(global_explanation, wrapped_model, dataset=x_test)